## ハイパーパラメータチューニング

sprint18-19のハイパーパラメータチューニング部分のノートブックです。

## optimizerのチューニング

In [141]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = '../input/stage1_train/'
TEST_PATH = '../input/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [142]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [143]:
# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

Getting and resizing train images and masks ... 


100%|██████████| 670/670 [01:24<00:00,  7.96it/s]

Getting and resizing test images ... 



100%|██████████| 65/65 [00:00<00:00, 92.03it/s]

Done!


In [54]:
# IoU metric を定義
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

compile(self, optimizer, loss, metrics=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)

In [127]:
# U-Net model　構築
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (s)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])

from keras import optimizers
params={}

params['sgd'] = optimizers.SGD(lr=0.01, clipvalue=0.5)
params['Adam'] =optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
params['RMSprop'] = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
params['Adagrad'] = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
params['Adadelta'] = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
params['Nadam'] = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
for key in params:
    model.compile(optimizer=key, loss='binary_crossentropy', metrics=[mean_iou])
    #model.summary()
    # Fit model
    earlystopper = EarlyStopping(patience=3, verbose=0)
    checkpointer = ModelCheckpoint('model-dsbowl2018-1'+key+'.h5', verbose=1, save_best_only=True)
    results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=8, epochs=30, verbose=1,
                        callbacks=[earlystopper, checkpointer])

Train on 603 samples, validate on 67 samples
Epoch 1/30
603/603 [==============================] - 45s 74ms/step - loss: 0.6255 - mean_iou: 0.4159 - val_loss: 0.5559 - val_mean_iou: 0.4237

Epoch 00001: val_loss improved from inf to 0.55586, saving model to model-dsbowl2018-1sgd.h5
Epoch 2/30
603/603 [==============================] - 40s 67ms/step - loss: 0.5121 - mean_iou: 0.4227 - val_loss: 0.4595 - val_mean_iou: 0.4236

Epoch 00002: val_loss improved from 0.55586 to 0.45949, saving model to model-dsbowl2018-1sgd.h5
Epoch 3/30
603/603 [==============================] - 38s 63ms/step - loss: 0.4542 - mean_iou: 0.4233 - val_loss: 0.4266 - val_mean_iou: 0.4236

Epoch 00003: val_loss improved from 0.45949 to 0.42655, saving model to model-dsbowl2018-1sgd.h5
Epoch 4/30
603/603 [==============================] - 38s 63ms/step - loss: 0.4394 - mean_iou: 0.4236 - val_loss: 0.4146 - val_mean_iou: 0.4236

Epoch 00004: val_loss improved from 0.42655 to 0.41459, saving model to model-dsbowl2018


Epoch 00003: val_loss improved from 0.07186 to 0.06714, saving model to model-dsbowl2018-1Adadelta.h5
Epoch 4/30
603/603 [==============================] - 36s 59ms/step - loss: 0.0654 - mean_iou: 0.8696 - val_loss: 0.0697 - val_mean_iou: 0.8698

Epoch 00004: val_loss did not improve from 0.06714
Epoch 5/30
603/603 [==============================] - 37s 61ms/step - loss: 0.0660 - mean_iou: 0.8703 - val_loss: 0.0718 - val_mean_iou: 0.8699

Epoch 00005: val_loss did not improve from 0.06714
Epoch 6/30
603/603 [==============================] - 40s 67ms/step - loss: 0.0644 - mean_iou: 0.8703 - val_loss: 0.0653 - val_mean_iou: 0.8707

Epoch 00006: val_loss improved from 0.06714 to 0.06527, saving model to model-dsbowl2018-1Adadelta.h5
Epoch 7/30
603/603 [==============================] - 40s 66ms/step - loss: 0.0655 - mean_iou: 0.8703 - val_loss: 0.0662 - val_mean_iou: 0.8708

Epoch 00007: val_loss did not improve from 0.06527
Epoch 8/30
603/603 [==============================] - 38s 63ms

以上の学習から各optimizerのlossと評価関数の値は下記の通り。

- SGD : val_loss  0.21166  val_mean_iou: 0.4693
- Adam : val_loss  0.07101 val_mean_iou: 0.8139
- Adagrad : val_loss 0.06724 val_mean_iou: 0.8648
- Adadelta : val_loss 0.06985 val_mean_iou: 0.8804
- Nadam : val_loss 0.06679  val_mean_iou: 0.8867  

 Nadamが一番精度がよかった。